In [1]:
!pip install pyedflib

In [2]:
import glob as glob
import os
import numpy as np
import mne 
import shutil
from mne.datasets import eegbci
from mne.datasets import sleep_physionet
import matplotlib.pyplot as plt
import pandas as pd
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs,
                               create_eog_epochs)
import torch
import numpy as np
import platform, os, re, multiprocessing
import pyedflib
import time  # Import the time module
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import platform, os, re, multiprocessing
import pandas as pd
import pyedflib
import mne
import matplotlib.pyplot as plt
import time  # Import the time module
import os
import mne
import warnings
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.pipeline import make_pipeline
import glob as glob
import numpy as np
import mne
import shutil
from mne.datasets import eegbci
from mne.datasets import sleep_physionet
import pandas as pd
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs,
                               create_eog_epochs)
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from datetime import timedelta
from mne.datasets import sample
from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA, FastICA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [3]:
import glob
import os

path = "/home/Duchuy220602/thesis/physionet.org/files/capslpdb/1.0.0"
pattern = "brux*.edf"

raw_list = []

for file_path in glob.glob(os.path.join(path, pattern)):
    raw_list.append(file_path)
    
# Define the desired sampling frequency
desired_sfreq = 128

# Read each file and append to a list of Raw objects
raw_objects = []
for file in raw_list:
    raw = mne.io.read_raw_edf(file, preload=True, verbose=0, infer_types=True)
    raw_objects.append(raw)

# Resample each Raw object to the desired sampling frequency
for raw_obj in raw_objects:
    raw_obj.resample(desired_sfreq)

print(raw_objects)

/tmp/ipykernel_10275/3686919123.py:18: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_edf(file, preload=True, verbose=0, infer_types=True)


[<RawEDF | brux2.edf, 18 x 4009088 (31321.0 s), ~550.6 MB, data loaded>, <RawEDF | brux1.edf, 18 x 1835648 (14341.0 s), ~252.1 MB, data loaded>]


In [4]:
path_csv = "/home/Duchuy220602/thesis/file_csv"
file_csv = "brux*.csv"

csv_list = []

for file_path in glob.glob(os.path.join(path_csv, file_csv)):
    csv_list.append(file_path)

for i in range(len(raw_objects)):
    raw = raw_objects[i]
    csv = csv_list[i]

    df = pd.read_csv(csv)

    # Convert the 'Time [hh:mm:ss]' column to a time format
    df['Time [hh:mm:ss]'] = pd.to_datetime(df['Time [hh:mm:ss]'], format='%H:%M:%S').dt.time

    # Extract the necessary columns from your DataFrame
    duration = df['Duration[s]']
    description = df['Sleep Stage']

    # Convert onset times to seconds
    onset = list(range(0, len(df)))  # Replace with your actual onset times

    # Create MNE Annotations
    my_annot = mne.Annotations(onset=onset, duration=duration, description=description)

    # Set annotations on the copied EEG data
    raw.set_annotations(my_annot)

    # Print the annotations
    print(raw.annotations)

<Annotations | 1418 segments: MT (8), REM (179), S1 (130), S2 (691), S3 ...>
<Annotations | 1511 segments: MT (3), R (207), S1 (162), S2 (482), S3 ...>


In [5]:
# If you want to concatenate the Raw objects into a single object
raw = mne.concatenate_raws(raw_objects)

In [7]:
# Print the concatenated annotations
print(raw.annotations)

<Annotations | 2931 segments: BAD boundary (1), EDGE boundary (1), MT ...>


In [8]:
# raw.save('/home/Duchuy220602/thesis/modify_file/brux_concatenated_raw.fif', overwrite=True)

Writing /home/Duchuy220602/thesis/modify_file/brux_concatenated_raw.fif
Closing /home/Duchuy220602/thesis/modify_file/brux_concatenated_raw.fif
[done]
